In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('dataset/total_preprocessed.csv')

In [3]:
df

,Total Length of Bwd Packet,Fwd Packet Length Min,Fwd Packet Length Mean,Bwd Packet Length Max,Bwd Packet Length Min,Flow Bytes/s,Flow Packets/s,Flow IAT Min,Fwd IAT Total,Fwd IAT Std,...,FIN Flag Count_3,RST Flag Count_0,RST Flag Count_1,RST Flag Count_2,URG Flag Count_0,URG Flag Count_1,URG Flag Count_2,ECE Flag Count_0,ECE Flag Count_1,ECE Flag Count_2
0,6143.0,0.0,56.692308,4104.0,0.0,2.819441e+05,901.565445,1.0,19947.0,2.328742e+03,...,False,True,False,False,True,False,False,True,False,False
1,34635.0,0.0,35.047619,15048.0,0.0,1.190181e+06,1110.400754,1.0,25441.0,2.259276e+03,...,False,True,False,False,True,False,False,True,False,False
2,34635.0,0.0,38.736842,16406.0,0.0,1.322478e+06,1196.440589,1.0,22381.0,2.038627e+03,...,False,True,False,False,True,False,False,True,False,False
3,0.0,24.0,24.000000,0.0,0.0,6.030151e+03,502.512563,3980.0,0.0,0.000000e+00,...,False,True,False,False,True,False,False,True,False,False
4,6169.0,0.0,65.454545,4104.0,0.0,1.704308e+04,51.953064,2.0,399628.0,1.138181e+05,...,False,True,False,False,True,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268846,0.0,0.0,4.500000,0.0,0.0,2.526572e+00,1.263286,2.0,7121185.0,4.065440e+06,...,False,True,False,False,True,False,False,True,False,False
268847,0.0,0.0,172.333333,0.0,0.0,4.889814e+03,56.748321,139.0,4524.0,2.015254e+03,...,False,True,False,False,True,False,False,True,False,False
268848,0.0,7.0,7.000000,0.0,0.0,1.924134e+03,549.752611,3638.0,0.0,0.000000e+00,...,False,False,True,False,True,False,False,True,False,False
268849,0.0,0.0,172.333333,0.0,0.0,4.901450e+03,56.883361,51.0,3479.0,1.832114e+03,...,False,True,False,False,True,False,False,True,False,False
